# Bottle Processing
Author: Andrew Reed

### Motivation:
Independent verification of the suite of physical and chemical observations provided by OOI are critical for the observations to be of use for scientifically valid investigations. Consequently, CTD casts and Niskin water samples are made during deployment and recovery of OOI platforms, vehicles, and instrumentation. The water samples are subsequently analyzed by independent labs for  comparison with the OOI telemetered and recovered data.

However, currently the water sample data routinely collected and analyzed as part of the OOI program are not available in a standardized format which maps the different chemical analyses to the physical measurements taken at bottle closure. Our aim is to make these physical and chemical analyses of collected water samples available to the end-user in a standardized format for easy comprehension and use, while maintaining the source data files. 

### Approach:
Generating a summary of the water sample analyses involves preprocessing and concatenating multiple data sources, and accurately matching samples with each other. To do this, I first preprocess the ctd casts to generate bottle (.btl) files using the SeaBird vendor software following the SOP available on Alfresco. 

Next, the bottle files are parsed using python code and the data renamed following SeaBird's naming guide. This creates a series of individual cast summary (.sum) files. These files are then loaded into pandas dataframes, appended to each other, and exported as a csv file containing all of the bottle data in a single data file.

### Data Sources/Software:

* **sbe_name_map**: This is a spreadsheet which maps the short names generated by the SeaBird SBE DataProcessing Software to the associated full names. The name mapping originates from SeaBird's SBE DataProcessing support documentation.

* **Alfresco**: The Alfresco CMS for OOI at alfresco.oceanobservatories.org is the source of the ctd hex, xmlcon, and psa files necessary for generating the bottle files needed to create the sample summary sheet.

* **SBEDataProcessing-Win32**: SeaBird vendor software for processing the raw ctd files and generating the .btl files.


In [1]:
# Import packages used in this notebook
import os, sys, re
import pandas as pd
import numpy as np

In [2]:
# Load the name mapping for the column names
# Specifiy the local directory
# sbe_name_map = pd.read_excel('/media/andrew/OS/Users/areed/Documents/OOI-CGSN/QAQC_Sandbox/Reference_Files/seabird_ctd_name_map.xlsx')
sbe_name_map = pd.read_excel('C:/Users/areed/Documents/OOI-CGSN/QAQC_Sandbox/Reference_Files/seabird_ctd_name_map.xlsx')

In [3]:
# Specify the local directory where the bottle (.btl) files are stored for a particular cruise
dirpath = 'C:/Users/areed/Documents/OOI-CGSN/QAQC_Sandbox/Ship_data/Irminger/Irminger-5/ctd/'
#datapath = '/media/andrew/OS/Users/areed/Documents/OOI-CGSN/QAQC_Sandbox/Ship_data/Irminger/Irminger-5/Data/CTD/'

In [4]:
# Just checks and prints that there are parseable btl files in the specified directory
for file in os.listdir(dirpath):
    if '.btl' in file:
        print(file)

ar30-03001.btl
ar30-03002b.btl
ar30-03003.btl
ar30-03004.btl
ar30-03005.btl
ar30-03006.btl
ar30-03007.btl
ar30-03008.btl
ar30-03009.btl
ar30-03010.btl
ar30-03011.btl
ar30-03012.btl
ar30-03013.btl
ar30-03014.btl
ar30-03015.btl
ar30-03016.btl
ar30-03017.btl
ar30-03018.btl
ar30-03019.btl
ar30-03020.btl
ar30-03021.btl
ar30-03022.btl
ar30-03023.btl


In [5]:
# Parse the data for the start_time
def parse_header(header):
    """
    Parse the header of bottle (.btl) files to get critical information
    for the summary spreadsheet.
    
    Args:
        header - an object containing the header of the bottle file as a list of
            strings, split at the newline.
    Returns:
        hdr - a dictionary object containing the start_time, filename, latitude,
            longitude, and cruise id.
    """
    hdr = {}
    for line in header:
        if 'start_time' in line.lower():
            start_time = pd.to_datetime(re.split('= |\[',line)[1])
            hdr.update({'Start Time':start_time.strftime('%Y-%m-%dT%H:%M:%SZ')})
        elif 'filename' in line.lower():
            hex_name = re.split('=',line)[1].strip()
            hdr.update({'Hex name':hex_name})
        elif 'latitude' in line.lower():
            start_lat = re.split('=',line)[1].strip()
            hdr.update({'Start Latitude':start_lat})
        elif 'longitude' in line.lower():
            start_lon = re.split('=',line)[1].strip()
            hdr.update({'Start Longitude':start_lon})
        elif 'cruise id' in line.lower():
            cruise_id = re.split(':',line)[1].strip()
            hdr.update({'Cruise ID':cruise_id})
        else:
            pass
    
    return hdr
        

In [6]:
# Now write a function to autopopulate the bottle summary sample sheet
sbe_name_map;
dirpath;

# Now 
files = [x for x in os.listdir(dirpath) if '.btl' in x]
for filename in files:
    filepath = os.path.abspath(dirpath+filename)
    
    # Load the raw content into memory
    with open(filepath) as file:
        content = file.readlines()
    content = [x.strip() for x in content]
    
    # Now parse the file content
    header = []
    columns = []
    data = []
    for line in content:
        if line.startswith('*') or line.startswith('#'):
            header.append(line)
        else:
            try:
                float(line[0])
                data.append(line)
            except:
                columns.append(line)
    
    # Parse the header
    hdr = parse_header(header)
    
    # Parse the column identifiers
    column_dict = {}
    for line in columns:
        for i,x in enumerate(line.split()):
            try:
                column_dict[i] = column_dict[i] + ' ' + x
            except:
                column_dict.update({i:x})
    
    # Parse the bottle data based on the column header locations
    data_dict = {x:[] for x in column_dict.keys()}

    for line in data:
        if line.endswith('(avg)'):
            values = list(filter(None,re.split('  |\t', line) ) )
            for i,x in enumerate(values):
                data_dict[i].append(x)
        elif line.endswith('(sdev)'):
            values = list(filter(None,re.split('  |\t', line) ) )
            data_dict[1].append(values[0])
        else:
            pass
            
    data_dict[1] = [' '.join(item) for item in zip(data_dict[1][::2],data_dict[1][1::2])]
    
    # With the parsed data and column names, match up the data and column
    # based on the location
    results = {}
    for key,item in column_dict.items():
        values = data_dict[key]
        results.update({item:values})
        
    # Put the results into a dataframe
    df = pd.DataFrame.from_dict(results)
        
    # Now add the parsed info from the header files into the dataframe
    for key,item in hdr.items():
        df[key] = item
        
    # Get the cast number
    cast = filename[filename.index('.')-3:filename.index('.')]
    df['Cast'] = cast
    
    # Generate a filename for the summary file
    outname = filename.split('.')[0] + '.sum'
    
    # Save the results
    df.to_csv(dirpath+outname)

In [7]:
df

,Bottle Position,Date Time,PrDM,DepSM,Latitude,Longitude,T090C,T190C,C0S/m,C1S/m,...,Sbeox0ML/L,OxsolML/L,CStarAt0,CStarTr0,Hex name,Start Latitude,Start Longitude,Cruise ID,Start Time,Cast
0,1,Jun 22 2018 20:15:22,154.275,152.738,62.05857,-32.23057,5.1590,5.1572,3.359006,3.359001,...,6.1666,7.05095,24.5949,0.2146 (avg),D:\Data\ar30-03023.hex,62 03.51 N,032 13.91 W,ar30-03,2018-06-22T20:07:00Z,023
1,2,Jun 22 2018 20:17:53,103.629,102.610,62.05862,-32.23032,5.1890,5.1874,3.357294,3.357300,...,6.5501,7.04710,55.2620,-0.1631 (avg),D:\Data\ar30-03023.hex,62 03.51 N,032 13.91 W,ar30-03,2018-06-22T20:07:00Z,023
2,3,Jun 22 2018 20:20:58,43.075,42.657,62.05860,-32.23016,6.9097,6.9319,3.511655,3.513982,...,6.7519,6.77019,22.9705,0.3219 (avg),D:\Data\ar30-03023.hex,62 03.51 N,032 13.91 W,ar30-03,2018-06-22T20:07:00Z,023
3,4,Jun 22 2018 20:22:42,25.871,25.621,62.05861,-32.23018,7.2552,7.2505,3.543044,3.542667,...,6.7507,6.71676,18.9658,0.8727 (avg),D:\Data\ar30-03023.hex,62 03.51 N,032 13.91 W,ar30-03,2018-06-22T20:07:00Z,023
4,5,Jun 22 2018 20:24:17,17.712,17.542,62.05862,-32.23026,7.2777,7.2778,3.544142,3.544219,...,6.8086,6.71361,18.9499,0.8783 (avg),D:\Data\ar30-03023.hex,62 03.51 N,032 13.91 W,ar30-03,2018-06-22T20:07:00Z,023
5,6,Jun 22 2018 20:26:11,2.963,2.934,62.05860,-32.23042,7.2822,7.2818,3.542571,3.542688,...,6.8793,6.71357,21.1395,0.5070 (avg),D:\Data\ar30-03023.hex,62 03.51 N,032 13.91 W,ar30-03,2018-06-22T20:07:00Z,023


In [8]:
for file in os.listdir(dirpath):
    if '.sum' in file:
        print(file)

ar30-03001.sum
ar30-03002b.sum
ar30-03003.sum
ar30-03004.sum
ar30-03005.sum
ar30-03006.sum
ar30-03007.sum
ar30-03008.sum
ar30-03009.sum
ar30-03010.sum
ar30-03011.sum
ar30-03012.sum
ar30-03013.sum
ar30-03014.sum
ar30-03015.sum
ar30-03016.sum
ar30-03017.sum
ar30-03018.sum
ar30-03019.sum
ar30-03020.sum
ar30-03021.sum
ar30-03022.sum
ar30-03023.sum


In [9]:
# Now, for each "summary" file, load and append to each other
df = pd.DataFrame()
for file in os.listdir(dirpath):
    if '.sum' in file:
        df = df.append(pd.read_csv(dirpath+file))
    else:
        pass
            
        

In [10]:
df

,Unnamed: 0,Bottle Position,Date Time,PrDM,DepSM,Latitude,Longitude,T090C,T190C,C0S/m,...,Sbeox0ML/L,OxsolML/L,CStarAt0,CStarTr0,Hex name,Start Latitude,Start Longitude,Cruise ID,Start Time,Cast
0,0,1,Jun 06 2018 09:26:42,148.209,146.727,62.77390,-28.65319,6.1764,6.1704,3.455169,...,6.2855,6.88271,19.0368,0.8578 (avg),D:\Data\ar30-03001.hex,62 46.29 N,028 39.40 W,ar30-03,2018-06-06T08:16:04Z,1
1,1,2,Jun 06 2018 09:31:25,105.773,104.726,62.77408,-28.65296,6.4277,6.4273,3.476623,...,6.3981,6.84254,19.1611,0.8311 (avg),D:\Data\ar30-03001.hex,62 46.29 N,028 39.40 W,ar30-03,2018-06-06T08:16:04Z,1
2,2,3,Jun 06 2018 09:36:05,40.563,40.167,62.77418,-28.65280,6.7659,6.7668,3.505544,...,6.6604,6.78899,21.4223,0.4769 (avg),D:\Data\ar30-03001.hex,62 46.29 N,028 39.40 W,ar30-03,2018-06-06T08:16:04Z,1
3,3,4,Jun 06 2018 09:38:14,23.395,23.168,62.77420,-28.65278,7.1446,7.1685,3.540467,...,7.0707,6.72998,26.6674,0.1282 (avg),D:\Data\ar30-03001.hex,62 46.29 N,028 39.40 W,ar30-03,2018-06-06T08:16:04Z,1
4,4,5,Jun 06 2018 09:39:31,15.493,15.343,62.77426,-28.65268,7.3552,7.3551,3.558928,...,7.0591,6.69807,28.1697,0.1376 (avg),D:\Data\ar30-03001.hex,62 46.29 N,028 39.40 W,ar30-03,2018-06-06T08:16:04Z,1
5,5,6,Jun 06 2018 09:41:34,2.739,2.713,62.77432,-28.65258,7.3695,7.3698,3.559666,...,7.0726,6.69589,22.2056,0.3952 (avg),D:\Data\ar30-03001.hex,62 46.29 N,028 39.40 W,ar30-03,2018-06-06T08:16:04Z,1
6,6,7,Jun 06 2018 09:41:42,2.972,2.943,62.77432,-28.65257,7.3710,7.3712,3.559819,...,7.0735,6.69567,22.1383,0.3963 (avg),D:\Data\ar30-03001.hex,62 46.29 N,028 39.40 W,ar30-03,2018-06-06T08:16:04Z,1
0,0,1,Jun 06 2018 14:24:52,155.360,153.807,62.50420,-29.79834,6.5441,6.5448,3.491486,...,6.2735,6.82320,16.6596,1.5555 (avg),D:\Data\ar30-03002b.hex,62 30.13 N,029 47.90 W,ar30-03,2018-06-06T14:12:41Z,02b
1,1,2,Jun 06 2018 14:27:57,99.927,98.942,62.50471,-29.79834,6.6319,6.6321,3.496320,...,6.4129,6.80975,17.7897,1.1718 (avg),D:\Data\ar30-03002b.hex,62 30.13 N,029 47.90 W,ar30-03,2018-06-06T14:12:41Z,02b
2,2,3,Jun 06 2018 14:31:07,39.452,39.068,62.50523,-29.79833,6.8601,6.8605,3.514970,...,6.5663,6.77392,17.3674,1.3014 (avg),D:\Data\ar30-03002b.hex,62 30.13 N,029 47.90 W,ar30-03,2018-06-06T14:12:41Z,02b


In [11]:
sbe_name_map['Short Name'].apply(lambda x: str(x).lower());

In [12]:
# Rename the column title using the sbe_name_mapping 
for colname in list(df.columns.values):
    try:
        fullname = list(sbe_name_map[sbe_name_map['Short Name'].apply(lambda x: str(x).lower() == colname.lower()) == True]['Full Name'])[0]
        df.rename({colname:fullname},axis='columns',inplace=True)
    except:
        pass

In [13]:
df

,Unnamed: 0,Bottle Position,Date Time,"Pressure, Digiquartz [db]","Depth [salt water, m]",Latitude [deg],Longitude [deg],"Temperature [ITS-90, deg C]","Temperature, 2 [ITS-90, deg C]",Conductivity [S/m],...,"Oxygen, SBE 43 [ml/l]","Oxygen Saturation, Garcia & Gordon [ml/l]","Beam Attenuation, WET Labs C-Star [1/m]","Beam Transmission, WET Labs C-Star [%]",Hex name,Start Latitude,Start Longitude,Cruise ID,Start Time,Cast
0,0,1,Jun 06 2018 09:26:42,148.209,146.727,62.77390,-28.65319,6.1764,6.1704,3.455169,...,6.2855,6.88271,19.0368,0.8578 (avg),D:\Data\ar30-03001.hex,62 46.29 N,028 39.40 W,ar30-03,2018-06-06T08:16:04Z,1
1,1,2,Jun 06 2018 09:31:25,105.773,104.726,62.77408,-28.65296,6.4277,6.4273,3.476623,...,6.3981,6.84254,19.1611,0.8311 (avg),D:\Data\ar30-03001.hex,62 46.29 N,028 39.40 W,ar30-03,2018-06-06T08:16:04Z,1
2,2,3,Jun 06 2018 09:36:05,40.563,40.167,62.77418,-28.65280,6.7659,6.7668,3.505544,...,6.6604,6.78899,21.4223,0.4769 (avg),D:\Data\ar30-03001.hex,62 46.29 N,028 39.40 W,ar30-03,2018-06-06T08:16:04Z,1
3,3,4,Jun 06 2018 09:38:14,23.395,23.168,62.77420,-28.65278,7.1446,7.1685,3.540467,...,7.0707,6.72998,26.6674,0.1282 (avg),D:\Data\ar30-03001.hex,62 46.29 N,028 39.40 W,ar30-03,2018-06-06T08:16:04Z,1
4,4,5,Jun 06 2018 09:39:31,15.493,15.343,62.77426,-28.65268,7.3552,7.3551,3.558928,...,7.0591,6.69807,28.1697,0.1376 (avg),D:\Data\ar30-03001.hex,62 46.29 N,028 39.40 W,ar30-03,2018-06-06T08:16:04Z,1
5,5,6,Jun 06 2018 09:41:34,2.739,2.713,62.77432,-28.65258,7.3695,7.3698,3.559666,...,7.0726,6.69589,22.2056,0.3952 (avg),D:\Data\ar30-03001.hex,62 46.29 N,028 39.40 W,ar30-03,2018-06-06T08:16:04Z,1
6,6,7,Jun 06 2018 09:41:42,2.972,2.943,62.77432,-28.65257,7.3710,7.3712,3.559819,...,7.0735,6.69567,22.1383,0.3963 (avg),D:\Data\ar30-03001.hex,62 46.29 N,028 39.40 W,ar30-03,2018-06-06T08:16:04Z,1
0,0,1,Jun 06 2018 14:24:52,155.360,153.807,62.50420,-29.79834,6.5441,6.5448,3.491486,...,6.2735,6.82320,16.6596,1.5555 (avg),D:\Data\ar30-03002b.hex,62 30.13 N,029 47.90 W,ar30-03,2018-06-06T14:12:41Z,02b
1,1,2,Jun 06 2018 14:27:57,99.927,98.942,62.50471,-29.79834,6.6319,6.6321,3.496320,...,6.4129,6.80975,17.7897,1.1718 (avg),D:\Data\ar30-03002b.hex,62 30.13 N,029 47.90 W,ar30-03,2018-06-06T14:12:41Z,02b
2,2,3,Jun 06 2018 14:31:07,39.452,39.068,62.50523,-29.79833,6.8601,6.8605,3.514970,...,6.5663,6.77392,17.3674,1.3014 (avg),D:\Data\ar30-03002b.hex,62 30.13 N,029 47.90 W,ar30-03,2018-06-06T14:12:41Z,02b


In [14]:
df.shape

(340, 23)

In [15]:
df.drop(columns='Unnamed: 0',inplace=True)

In [16]:
df

,Bottle Position,Date Time,"Pressure, Digiquartz [db]","Depth [salt water, m]",Latitude [deg],Longitude [deg],"Temperature [ITS-90, deg C]","Temperature, 2 [ITS-90, deg C]",Conductivity [S/m],"Conductivity, 2 [S/m]",...,"Oxygen, SBE 43 [ml/l]","Oxygen Saturation, Garcia & Gordon [ml/l]","Beam Attenuation, WET Labs C-Star [1/m]","Beam Transmission, WET Labs C-Star [%]",Hex name,Start Latitude,Start Longitude,Cruise ID,Start Time,Cast
0,1,Jun 06 2018 09:26:42,148.209,146.727,62.77390,-28.65319,6.1764,6.1704,3.455169,3.454611,...,6.2855,6.88271,19.0368,0.8578 (avg),D:\Data\ar30-03001.hex,62 46.29 N,028 39.40 W,ar30-03,2018-06-06T08:16:04Z,1
1,2,Jun 06 2018 09:31:25,105.773,104.726,62.77408,-28.65296,6.4277,6.4273,3.476623,3.476591,...,6.3981,6.84254,19.1611,0.8311 (avg),D:\Data\ar30-03001.hex,62 46.29 N,028 39.40 W,ar30-03,2018-06-06T08:16:04Z,1
2,3,Jun 06 2018 09:36:05,40.563,40.167,62.77418,-28.65280,6.7659,6.7668,3.505544,3.505613,...,6.6604,6.78899,21.4223,0.4769 (avg),D:\Data\ar30-03001.hex,62 46.29 N,028 39.40 W,ar30-03,2018-06-06T08:16:04Z,1
3,4,Jun 06 2018 09:38:14,23.395,23.168,62.77420,-28.65278,7.1446,7.1685,3.540467,3.542229,...,7.0707,6.72998,26.6674,0.1282 (avg),D:\Data\ar30-03001.hex,62 46.29 N,028 39.40 W,ar30-03,2018-06-06T08:16:04Z,1
4,5,Jun 06 2018 09:39:31,15.493,15.343,62.77426,-28.65268,7.3552,7.3551,3.558928,3.558911,...,7.0591,6.69807,28.1697,0.1376 (avg),D:\Data\ar30-03001.hex,62 46.29 N,028 39.40 W,ar30-03,2018-06-06T08:16:04Z,1
5,6,Jun 06 2018 09:41:34,2.739,2.713,62.77432,-28.65258,7.3695,7.3698,3.559666,3.559626,...,7.0726,6.69589,22.2056,0.3952 (avg),D:\Data\ar30-03001.hex,62 46.29 N,028 39.40 W,ar30-03,2018-06-06T08:16:04Z,1
6,7,Jun 06 2018 09:41:42,2.972,2.943,62.77432,-28.65257,7.3710,7.3712,3.559819,3.559834,...,7.0735,6.69567,22.1383,0.3963 (avg),D:\Data\ar30-03001.hex,62 46.29 N,028 39.40 W,ar30-03,2018-06-06T08:16:04Z,1
0,1,Jun 06 2018 14:24:52,155.360,153.807,62.50420,-29.79834,6.5441,6.5448,3.491486,3.491553,...,6.2735,6.82320,16.6596,1.5555 (avg),D:\Data\ar30-03002b.hex,62 30.13 N,029 47.90 W,ar30-03,2018-06-06T14:12:41Z,02b
1,2,Jun 06 2018 14:27:57,99.927,98.942,62.50471,-29.79834,6.6319,6.6321,3.496320,3.496352,...,6.4129,6.80975,17.7897,1.1718 (avg),D:\Data\ar30-03002b.hex,62 30.13 N,029 47.90 W,ar30-03,2018-06-06T14:12:41Z,02b
2,3,Jun 06 2018 14:31:07,39.452,39.068,62.50523,-29.79833,6.8601,6.8605,3.514970,3.515027,...,6.5663,6.77392,17.3674,1.3014 (avg),D:\Data\ar30-03002b.hex,62 30.13 N,029 47.90 W,ar30-03,2018-06-06T14:12:41Z,02b


In [17]:
df.shape

(340, 22)

In [18]:
df.to_csv(dirpath+'Irminger-5_Summary.csv')

In [ ]:
os.listdir(dirpath)